In [ ]:
Use [ReportServer]

In [ ]:
-- Connection strings of all SSRS Shared Datasources
;WITH XMLNAMESPACES  -- XML namespace def must be the first in with clause.
    (DEFAULT 'http://schemas.microsoft.com/sqlserver/reporting/2006/03/reportdatasource'
            ,'http://schemas.microsoft.com/SQLServer/reporting/reportdesigner'
     AS rd)
,SDS AS
    (SELECT SDS.name AS SharedDsName
           ,SDS.[Path]
           ,CONVERT(xml, CONVERT(varbinary(max), content)) AS DEF
     FROM dbo.[Catalog] AS SDS
     WHERE SDS.Type = 5)     -- 5 = Shared Datasource

SELECT CON.[Path]
      ,CON.SharedDsName
      ,CON.ConnString
FROM
    (SELECT SDS.[Path]
           ,SDS.SharedDsName
           ,DSN.value('ConnectString[1]', 'varchar(150)') AS ConnString
     FROM SDS
          CROSS APPLY 
          SDS.DEF.nodes('/DataSourceDefinition') AS R(DSN)
     ) AS CON
-- Optional filter:
-- WHERE CON.ConnString LIKE '%Initial Catalog%=%TFS%'
ORDER BY CON.[Path]
        ,CON.SharedDsName;

In [ ]:
-- Who has access to Report Server reports and objects.
select C.UserName, D.RoleName, D.Description, E.Path, E.Name 
from dbo.PolicyUserRole A 
   inner join dbo.Policies B on A.PolicyID = B.PolicyID 
   inner join dbo.Users C on A.UserID = C.UserID 
   inner join dbo.Roles D on A.RoleID = D.RoleID 
   inner join dbo.Catalog E on A.PolicyID = E.PolicyID 
order by C.UserName   

In [ ]:
-- Who access what in the last (60 days or server setting).
select * from ExecutionLog3 order by TimeStart DESC  

In [ ]:
-- All Reports last accessed  (60 days or server setting).
SELECT A.*,ISNULL(B.CNT,0) AS Usage, B.LastAccessed,
'http://localhost/Reports/Pages/Report.aspx?ItemPath='+dbo.ufn_UrlEncode(A.Directory) AS [URL]
FROM (
SELECT
 @@SERVERNAME AS ReportType
    ,CATALOG.[Path] AS Directory
	,CASE
	WHEN CATALOG.[Type] = 14 THEN 'Excel'
	WHEN CATALOG.[Type] = 13 THEN 'PowerBI'
	WHEN CATALOG.[Type] = 11 THEN 'KPI'
    WHEN CATALOG.[Type] = 8 THEN 'DataSetDataSource'
    WHEN CATALOG.[Type] = 7 THEN 'Report Part'    
    WHEN CATALOG.[Type] = 5 THEN 'Data Source'  
    WHEN CATALOG.[Type] = 3 THEN 'Other File Type'  
	WHEN CATALOG.[Type] = 2 THEN 'Report'
	WHEN CATALOG.[Type] = 1 THEN 'Folder'
	ELSE 'Other'
	END AS Dashboard
	 ,CATALOG.NAME AS ReportName
    ,DataSource.NAME datasource

FROM [dbo].[Catalog]
LEFT JOIN (
    SELECT [UserID]
        ,[UserName]
    FROM [dbo].[Users]
    ) AS Created ON CATALOG.CreatedByID = Created.UserID
LEFT JOIN (
    SELECT [UserID]
        ,[UserName]
    FROM [dbo].[Users]
    ) AS Modified ON CATALOG.ModifiedByID = Modified.UserID
left outer JOIN DataSource ON CATALOG.ItemID = DataSource.ItemID
LEFT outer JOIN CATALOG cat1 ON DataSource.Link = cat1.ItemID
WHERE CATALOG.[Type] NOT IN( 1,5) 
) AS A
LEFT JOIN (

SELECT ItemPath,c.Name, COUNT(*) AS CNT, MAX(TimeStart) AS LastAccessed 
FROM ExecutionLog3 AS e
LEFT JOIN Catalog c  
ON E.ItemPath = C.Path
GROUP BY ItemPath,Name

) AS B
ON A.ReportName = B.Name 


In [ ]:
-- List All Reports, Path, Datasource used
SELECT CATALOG.NAME
    ,CATALOG.[Path]
    ,DataSource.NAME datasource
    ,CATALOG.[Description]
    ,Created.UserName AS CreatedByUser
    ,CATALOG.[CreationDate]
    ,Modified.UserName AS ModifiedByUser
    ,CATALOG.[ModifiedDate]
FROM [dbo].[Catalog]
LEFT JOIN (
    SELECT [UserID]
        ,[UserName]
    FROM [dbo].[Users]
    ) AS Created ON CATALOG.CreatedByID = Created.UserID
LEFT JOIN (
    SELECT [UserID]
        ,[UserName]
    FROM [dbo].[Users]
    ) AS Modified ON CATALOG.ModifiedByID = Modified.UserID
left outer JOIN DataSource ON CATALOG.ItemID = DataSource.ItemID
LEFT outer JOIN CATALOG cat1 ON DataSource.Link = cat1.ItemID
WHERE CATALOG.[Type] = 2 
ORDER BY CATALOG.[ModifiedDate] desc, [Path] ,NAME

In [ ]:
-- Show the SQL queries inside the Reports (RDL)
WITH ItemContentBinaries AS
(
  SELECT
     ItemID,Name,[Type]
    ,CASE Type
       WHEN 2 THEN 'Report'
       WHEN 5 THEN 'Data Source'
       WHEN 7 THEN 'Report Part'
       WHEN 8 THEN 'Shared Dataset'
       ELSE 'Other'
     END AS TypeDescription
    ,CONVERT(varbinary(max),Content) AS Content
  FROM ReportServer.dbo.Catalog
  WHERE Type IN (2,5,7,8)
),
--The second CTE strips off the BOM if it exists...
ItemContentNoBOM AS
(
  SELECT
     ItemID,Name,[Type],TypeDescription
    ,CASE
       WHEN LEFT(Content,3) = 0xEFBBBF
         THEN CONVERT(varbinary(max),SUBSTRING(Content,4,LEN(Content)))
       ELSE
         Content
     END AS Content
  FROM ItemContentBinaries
)
--The old outer query is now a CTE to get the content in its xml form only...
,ItemContentXML AS
(
  SELECT
     ItemID,Name,[Type],TypeDescription
    ,CONVERT(xml,Content) AS ContentXML
 FROM ItemContentNoBOM
)
--now use the XML data type to extract the queries, and their command types and text....
SELECT
     ItemID,Name,[Type],TypeDescription,ContentXML
    ,ISNULL(Query.value('(./*:CommandType/text())[1]','nvarchar(1024)'),'Query') AS CommandType
    ,Query.value('(./*:CommandText/text())[1]','nvarchar(max)') AS CommandText
FROM ItemContentXML
--Get all the Query elements (The "*:" ignores any xml namespaces)
CROSS APPLY ItemContentXML.ContentXML.nodes('//*:Query') Queries(Query)

In [ ]:
-- List all ReportServer objects 
SELECT CATALOG.NAME
    ,CATALOG.[Path]
    ,DataSource.NAME datasource
    ,CATALOG.[Description]
    ,Created.UserName AS CreatedByUser
    ,CATALOG.[CreationDate]
    ,Modified.UserName AS ModifiedByUser
    ,CATALOG.[ModifiedDate]
	,CATALOG.[Type]
	,CASE
	WHEN CATALOG.[Type] = 14 THEN 'Excel'
	WHEN CATALOG.[Type] = 13 THEN 'PowerBI'
	WHEN CATALOG.[Type] = 11 THEN 'KPI'
    WHEN CATALOG.[Type] = 8 THEN 'DataSetDataSource'
    WHEN CATALOG.[Type] = 7 THEN 'Report Part'    
    WHEN CATALOG.[Type] = 5 THEN 'Data Source'  
    WHEN CATALOG.[Type] = 3 THEN 'Other File Type'  
	WHEN CATALOG.[Type] = 2 THEN 'Report'
	WHEN CATALOG.[Type] = 1 THEN 'Folder'
	ELSE 'Other'
	END AS [ObjectType]
FROM [dbo].[Catalog]
LEFT JOIN (
    SELECT [UserID]
        ,[UserName]
    FROM [dbo].[Users]
    ) AS Created ON CATALOG.CreatedByID = Created.UserID
LEFT JOIN (
    SELECT [UserID]
        ,[UserName]
    FROM [dbo].[Users]
    ) AS Modified ON CATALOG.ModifiedByID = Modified.UserID
left outer JOIN DataSource ON CATALOG.ItemID = DataSource.ItemID
LEFT outer JOIN CATALOG cat1 ON DataSource.Link = cat1.ItemID
--WHERE CATALOG.[Type] = 2 
ORDER BY CATALOG.[ModifiedDate] desc, [Path] ,NAME

In [ ]:
-- Export all Reports (RDL) from database
/*
-- Updated the code from here: http://vinay.inkeysolutions.com/2013/08/ssrs-download-all-rdl-files-from-report.html
-- Allow advanced options to be changed.
EXEC sp_configure 'show advanced options', 1
GO
 
-- Update the currently configured value for advanced options.
RECONFIGURE
GO
 
-- Enable xp_cmdshell
EXEC sp_configure 'xp_cmdshell', 1
GO
 
-- Update the currently configured value for xp_cmdshell
RECONFIGURE
GO
 
-- Disallow further advanced options to be changed.
EXEC sp_configure 'show advanced options', 0
GO
 
-- Update the currently configured value for advanced options.
RECONFIGURE
GO
*/

--Replace NULL with keywords of the ReportManager's Report Path, 
--if reports from any specific path are to be downloaded
DECLARE @FilterReportPath AS VARCHAR(500) = NULL; 
 
--Replace NULL with the keyword matching the Report File Name,
--if any specific reports are to be downloaded
DECLARE @FilterReportName AS VARCHAR(500) = NULL;
 
--Replace this path with the Server Location where you want the
--reports to be downloaded..
DECLARE @OutputPath AS VARCHAR(500) = 'C:\Reports';
 --SELECT Path, @OutputPath + REPLACE(REPLACE(REVERSE(SUBSTRING(REVERSE(Path), CHARINDEX('/',REVERSE(Path)), LEN(Path) - CHARINDEX('/',REVERSE(Path)) + 1)), '/','\'), '\\\\','\\') FolderPath
	--	FROM ReportServer.dbo.Catalog WITH(NOLOCK)

--Used to prepare the dynamic query
DECLARE @TSQL AS NVARCHAR(MAX);
DECLARE @DataPath NVARCHAR(500);
DECLARE @DirTree TABLE
    (
      subdirectory NVARCHAR(255) ,
      depth INT
    );
 
--Reset the OutputPath separator.
SET @OutputPath = REPLACE(@OutputPath, '\', '/');
 
--Simple validation of OutputPath; this can be changed as per ones need.
IF LTRIM(RTRIM(ISNULL(@OutputPath, ''))) = ''
    BEGIN
        SELECT  'Invalid Output Path';
    END;
ELSE
    BEGIN
   --Prepare the query for download.
   /*
   Please note the following points -
   1. The BCP command could be modified as per ones need. E.g. Providing UserName/Password, etc.
   2. Please update the SSRS Report Database name. Currently, it is set to default - [ReportServer]
   3. The BCP does not create missing Directories. So, additional logic could be implemented to handle that.
   4. SSRS stores the XML items (Report RDL and Data Source definitions) using the UTF-8 encoding. 
      It just so happens that UTF-8 Unicode strings do not NEED to have a BOM and in fact ideally would not have one. 
      However, you will see some report items in your SSRS that begin with a specific sequence of bytes (0xEFBBBF). 
      That sequence is the UTF-8 Byte Order Mark. It’s character representation is the following three characters, “ï»¿”. 
      While it is supported, it can cause problems with the conversion to XML, so it is removed.
   */

        DECLARE @itemID UNIQUEIDENTIFIER = NULL;
        DECLARE @currentFolderPath NVARCHAR(2000);

		
        IF ( OBJECT_ID('tempdb..#catalog') IS NOT NULL )
            DROP TABLE #catalog;
        SELECT * ,
                0 AS Processed
        INTO    #catalog
        FROM    [ReportServer].[dbo].[Catalog] CL
        WHERE   CL.[Type] 
					 = 2 --Report
					-- = 5 -- DataSource
                AND '/' + CL.[Path] + '/' LIKE COALESCE('%/%'
                                                        + @FilterReportPath
                                                        + '%/%',
                                                        '/' + CL.[Path] + '/')
                AND CL.Name LIKE COALESCE('%' + @FilterReportName + '%',
                                          CL.Name);
        WHILE EXISTS ( SELECT   1
                       FROM     #catalog
                       WHERE    Processed = 0 )
            BEGIN

                SELECT TOP 1
                        @itemID = ItemID ,
                        @currentFolderPath = @OutputPath
                        + REPLACE(REPLACE(REVERSE(SUBSTRING(REVERSE(Path),
                                                            CHARINDEX('/',
                                                              REVERSE(Path)),
                                                            LEN(Path)
                                                            - CHARINDEX('/',
                                                              REVERSE(Path))
                                                            + 1)), '/', '\'),
                                  '\\\\', '\\')
                FROM    #catalog
                WHERE   Processed = 0;

                SET @TSQL = STUFF(( SELECT  ';EXEC master..xp_cmdshell ''bcp " '
                                            + ' SELECT '
                                            + ' CONVERT(VARCHAR(MAX), '
                                            + '       CASE '
                                            + '         WHEN LEFT(C.Content,3) = 0xEFBBBF THEN STUFF(C.Content,1,3,'''''''') '
                                            + '         ELSE C.Content '
                                            + '       END) ' + ' FROM '
                                            + ' [ReportServer].[dbo].[Catalog] CL '
                                            + ' CROSS APPLY (SELECT CONVERT(VARBINARY(MAX),CL.Content) Content) C '
                                            + ' WHERE ' + ' CL.ItemID = '''''
                                            + CONVERT(VARCHAR(MAX), CL.ItemID)
                                            + ''''' " queryout "'
                                            + @currentFolderPath + ''
                                            + CL.Name + '.rdl" '
                                            + '-T -c -x'''
                                    FROM    #catalog CL
                                    WHERE   CL.ItemID = @itemID
                                  FOR
                                    XML PATH('')
                                  ), 1, 1, '');

                DELETE  FROM @DirTree;
                INSERT  INTO @DirTree
                        ( subdirectory ,
                          depth
                        )
                        EXEC master.sys.xp_dirtree @OutputPath;
				
	--			SELECT @currentFolderPath;
                IF NOT EXISTS ( SELECT  1
                                FROM    @DirTree
                                WHERE   subdirectory = @currentFolderPath )
                    EXEC master.dbo.xp_create_subdir @currentFolderPath;
  
  --              SELECT  @TSQL;
  
				--Execute the Dynamic Query
                EXEC sp_executesql @TSQL;

                UPDATE  #catalog
                SET     Processed = 1
                WHERE   ItemID = @itemID;

            END;
    END;

In [ ]:
-- List All Subscriptions
WITH
[Sub_Parameters] AS
    (SELECT 
        [SubscriptionID],
        [Parameters] = CONVERT(XML,a.[Parameters]),
        [ExtensionSettings] = CONVERT(XML,a.[ExtensionSettings])
     FROM [Subscriptions] a
    )

, [MySubscriptions] AS (
    SELECT --DISTINCT 
        [SubscriptionID],
        [ParameterName] = QUOTENAME(p.value('(Name)[1]', 'nvarchar(max)')),
        [ParameterValue] = p.value('(Value)[1]', 'nvarchar(max)')
    FROM [Sub_Parameters] a
    CROSS APPLY [Parameters].nodes('/ParameterValues/ParameterValue') t(p)
    UNION
    SELECT --DISTINCT 
        [SubscriptionID],
        [ExtensionSettingName] = QUOTENAME(e.value('(Name)[1]', 'nvarchar(max)')),
        [ExtensionSettingValue] = e.value('(Value)[1]', 'nvarchar(max)')
    FROM [Sub_Parameters] a
    CROSS APPLY [ExtensionSettings].nodes('/ParameterValues/ParameterValue') t(e)
    )

, [SubscriptionsAnalysis] AS (
    SELECT
        a.[SubscriptionID],
        a.[ParameterName],
        [ParameterValue] =
              (SELECT STUFF((SELECT [ParameterValue] + ', ' as [text()] 
               FROM [MySubscriptions] 
               WHERE [SubscriptionID] = a.[SubscriptionID] 
                   AND [ParameterName] = a.[ParameterName] FOR XML PATH('')),1, 0, '')+'')
    FROM [MySubscriptions] a
    GROUP BY a.[SubscriptionID],a.[ParameterName]
    )

SELECT
    a.[SubscriptionID],
    c.[UserName] AS [Owner], 
    b.[Name],
    b.[Path],
    a.[Locale], 
    a.[InactiveFlags], 
    d.[UserName] AS [Modified_by], 
    a.[ModifiedDate], 
    a.[Description], 
    a.[LastStatus], 
    a.[EventType], 
    a.[LastRunTime], 
    a.[DeliveryExtension],
    a.[Version],
    e.[ParameterName],
    LEFT(e.[ParameterValue],LEN(e.[ParameterValue])-1) as [ParameterValue],
    SUBSTRING(b.[PATH],2,LEN(b.[PATH])-(CHARINDEX('/',REVERSE(b.[PATH]))+1)) AS [ProjectName]
FROM [Subscriptions] a 
INNER JOIN [Catalog] AS b ON a.[Report_OID] = b.[ItemID]
LEFT OUTER JOIN [Users] AS c ON a.[OwnerID] = c.[UserID]
LEFT OUTER JOIN [Users] AS d ON a.[ModifiedByID] = d.[Userid]
LEFT OUTER JOIN [SubscriptionsAnalysis] AS e ON a.[SubscriptionID] = e.[SubscriptionID]
;

In [ ]:
-- PowerShell - Export all object types from report server db
--https://github.com/microsoft/ReportingServicesTools
--https://community.powerbi.com/t5/Report-Server/Where-Can-I-find-the-published-report-in-PowerBI-Server/m-p/397968#M5165
// ========== Powershell below this line ===========
<# .SYNOPSIS
      Export of all SSRS reports datasources and images
   .DESCRIPTION
      This PowerShell script exports all (or filtered) reports, data sources and images directly from the ReportServer database
      to a specified folder. For the file name the complete report path is used; for file name invalid characters are replaced with a -.
      Reports are exported with .rdl as extension, data sources with .rds and resources without any additional extension.
      Please change the "Configuration data" below to your enviroment.
      Works with SQL Server 2005 and higher versions in all editions.
      Requires SELECT permission on the ReportServer database.
   .NOTES
      Author  : Olaf Helper
      Requires: PowerShell Version 1.0, Ado.Net assembly
   .LINK
      GetSqlBinary: http://msdn.microsoft.com/en-us/library/system.data.sqlclient.sqldatareader.getsqlbinary.aspx
#>


# Configuration data
[string] $server   = "YourServerName";        # SQL Server Instance.
[string] $database = "ReportServer";        # ReportServer Database.
[string] $folder   = "C:\WorkRepo\Export_PBI_SSRS\";          # Path to export the reports to.

# Select-Statement for file name & blob data with filter.	
$sql = "SELECT	CT.[Path]
        ,CT.[Type]
		,ISNULL(cc.ContentType,'SSRS') as ContentType
        ,CONVERT(varbinary(max), cc.[Content]) AS PBI_BinaryContent
        ,CONVERT(varbinary(max), ct.[Content]) AS RDL_BinaryContent
FROM dbo.[Catalog] AS CT
		LEFT OUTER JOIN dbo.CatalogItemExtendedContent cc
			ON ct.ItemID = cc.ItemId
WHERE CT.[Type] IN (2, 8, 5,13)
	AND ISNULL(cc.ContentType,'CatalogItem') = 'CatalogItem'";		
 #       WHERE CT.[Type] IN (8)";

# Open ADO.NET Connection with Windows authentification.
$con = New-Object Data.SqlClient.SqlConnection;
$con.ConnectionString = "Data Source=$server;Initial Catalog=$database;Integrated Security=True;";
$con.Open();

Write-Output ((Get-Date -format yyyy-MM-dd-HH:mm:ss) + ": Started ...");

# New command and reader.
$cmd = New-Object Data.SqlClient.SqlCommand $sql, $con;
$rd = $cmd.ExecuteReader();

$invalids = [System.IO.Path]::GetInvalidFileNameChars();
# Looping through all selected datasets.
While ($rd.Read())
{
    Try
    {
        # Get the name and make it valid.
        $name = $rd.GetString(0);
		Write-Output "fetching $name"
        foreach ($invalid in $invalids)
           {    $name = $name.Replace($invalid, "-");    }

        If ($rd.GetInt32(1) -eq 2)
            {    $name = $name + ".rdl";    }
        ElseIf ($rd.GetInt32(1) -eq 5)
            {    $name = $name + ".rds";    }
        ElseIf ($rd.GetInt32(1) -eq 8)
            {    $name = $name + ".rsd";    }
        ElseIf ($rd.GetInt32(1) -eq 11)
            {    $name = $name + ".kpi";    }
        ElseIf ($rd.GetInt32(1) -eq 13)
	    {   $name = $name + ".pbix";    }
	
        Write-Output ((Get-Date -format yyyy-MM-dd-HH:mm:ss) + ": Exporting {0}" -f $name);

        $name = [System.IO.Path]::Combine($folder, $name);

        # New BinaryWriter; existing file will be overwritten.
        $fs = New-Object System.IO.FileStream ($name), Create, Write;
        $bw = New-Object System.IO.BinaryWriter($fs);

        # Read of complete Blob with GetSqlBinary
        if ($rd.GetString(2) -eq "SSRS") {
			$bt = $rd.GetSqlBinary(4).Value;
		} else{
			$bt = $rd.GetSqlBinary(3).Value;
		}
		
        $bw.Write($bt, 0, $bt.Length);
        $bw.Flush();
        $bw.Close();
        $fs.Close();
    }
    Catch
    {
        Write-Output ($_.Exception.Message)
    }
    Finally
    {
        $fs.Dispose();
    }
}

# Closing & Disposing all objects
$rd.Close();
$cmd.Dispose();
$con.Close();
$con.Dispose();

Write-Output ((Get-Date -format yyyy-MM-dd-HH:mm:ss) + ": Finished");